In [1]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results= 1, doc_content_chars_max= 250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki) 
wiki.name

'wikipedia'

In [3]:
## arxiv
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results= 1, doc_content_chars_max= 250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]   ### tools for running the agent.


In [6]:
### custom tools 
## rag tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter


In [9]:
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/agents/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
retriver = vectorstore.as_retriever()

In [10]:
from langchain.tools.retriever import create_retriever_tool

retri_tool = create_retriever_tool(retriver, name= 'langchain-agents-search', description= 'search langchain agents docs')

In [12]:
retri_tool.name

'langchain-agents-search'

In [13]:
tools.append(retri_tool)  ### add the retriever tool to the tools list



In [14]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\disha\\anaconda3\\envs\\qa-chatbot\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langchain-agents-search', description='search langchain agents docs', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000022E2E36FD00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022EAB6E3A

In [16]:
## run these tools in the agent
from dotenv import load_dotenv
import os
import openai
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name= 'Llama3-8b-8192')



In [18]:
### prompt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
## Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [21]:
## agent executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [22]:
agent_executor.invoke({"input": "What is the latest news on langchain agents?"})



> Entering new AgentExecutor chain...

Invoking: `langchain-agents-search` with `{'query': 'latest news on langchain agents'}`


Build an Agent | 🦜️🔗 LangChain

Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a se

{'input': 'What is the latest news on langchain agents?',
 'output': 'It seems that you have already called the tool for tool call id "call_a6yw" and it yielded the latest information on LangChain Agents. According to the output, LangChain Agents are complex systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. They can interact with other tools and have conversational memory, making them suitable for multi-turn chatbots. Additionally, LangChain is growing and hiring for multiple roles, and you can find more information on Agents in the LangGraph documentation.'}

In [23]:
agent_executor.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witBased on the result, it seems that machine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to new, unseen data. These algorithms can then be used to perform tasks such as classification, regression, clustering, and more.

Here's a possible text response:

Machine learning is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. These algorithms can be used for a wide range of applications, including image and speech recogniti

{'input': 'What is machine learning',
 'output': "Based on the result, it seems that machine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to new, unseen data. These algorithms can then be used to perform tasks such as classification, regression, clustering, and more.\n\nHere's a possible text response:\n\nMachine learning is a subfield of artificial intelligence that involves training algorithms to learn from data and make predictions or decisions without being explicitly programmed. These algorithms can be used for a wide range of applications, including image and speech recognition, natural language processing, and personalization. By analyzing large amounts of data, machine learning algorithms can identify patterns and relationships that would be difficult or impossible for humans to detect, making it a powerful tool in many areas of science and industry."}